In [1]:
#@title Perfil Aerodinàmic

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# Parameters of the grid
shape        = (400, 200)
xrange       = (-8, 8)
yrange       = (-3, 5)

# Parameters of the airfoil
mu    = -0.15 + 0.15j
cut_h = 0.07
V_inf = 3
R         = np.sqrt((1 - mu.real)**2 + mu.imag**2)
theta     = np.linspace(0, 2*np.pi, 100)
zeta_circ = mu + R * np.exp(1j * theta)
z_shape   = (zeta_circ + 1/zeta_circ)

# Define the complex plane grid
rez  = np.linspace(*xrange, shape[0])[:, None]
imz  = np.linspace(*yrange, shape[1])[None, :]
z    = (rez + 1j * imz)

# Inverse Joukowsky transform
def z2zeta(z):
    discriminant = np.sqrt(z**2 - 4)
    zeta = (z + discriminant) / 2
    in_unit_circle = (np.abs(zeta) <= 1)
    below_wing = (z.real**2 < 4) & (z.imag > 0) & \
              (z.imag < cut_h * (4 - z.real**2))
    bad_branch = below_wing ^ in_unit_circle
    zeta[bad_branch] -= discriminant[bad_branch]
    return zeta

# W = vx - i vy
def complex_velocity(z, alpha):
    zeta     = z2zeta(z)
    gamma = 4 * np.pi * V_inf * R * np.sin( alpha + np.arcsin(mu.imag / R) )
    W_tilde  = V_inf * np.exp(-1j * alpha)
    W_tilde += 1j * gamma / (2 * np.pi * (zeta - mu))
    W_tilde -= V_inf * R**2 * np.exp(1j * alpha) / (zeta - mu)**2
    W = W_tilde / (1 - 1/zeta**2)
    return W.conj()

# Bernoullis's principle
def pressure(z, alpha):
    st = complex_velocity(z, alpha)
    P = 0.5 * (V_inf**2 - np.abs(st)**2)
    return P

def plot_wing(alpha_deg = 20):
    
    alpha = alpha_deg * (np.pi / 180)
    rot = np.exp(1j * alpha)
    z_airfoil = z_shape * rot.conj()
    
    ax.cla()
    fig1.set_size_inches(10,5)
    ax.set_xlim(*xrange)
    ax.set_ylim(*yrange)
    ax.imshow(pressure(z * rot, alpha).T, origin = 'lower', cmap = 'jet', extent = xrange + yrange, vmin = -10, vmax = 7)
    W = complex_velocity(z * rot, alpha) / rot
    ax.streamplot(z.real.T, z.imag.T, W.real.T, W.imag.T, color = 'k', linewidth = 0.7)
    ax.fill(z_airfoil.real, z_airfoil.imag, color = 'white', zorder=2)
    
    return fig1

##### Do the plotting #####
fig1, ax = plt.subplots(figsize = (10,5))
plt.rc('font', size = 12)
plt.close()

interact(
    plot_wing,
    alpha_deg = FloatSlider(value=20, min=0, max=45, step=1,
        description="Angle d'atac (en graus)",
        style={'description_width': '150px', },
        layout={'width': '800px'},
        readout_format='.2f'),
);

interactive(children=(FloatSlider(value=20.0, description="Angle d'atac (en graus)", layout=Layout(width='800p…

In [2]:
#@title Turbulència

import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

shape = (200, 50)
iters = 3000
rho0  = 100
tau   = 0.55
frame_every = 30

x     = np.arange(shape[0])[:, None]
y     = np.arange(shape[1])[None, :]
curl  = np.zeros(shape + (iters//frame_every,)) 

D2Q9  = np.array([[di, dj, 4 / (9 * 4**(di**2 + dj**2))] for di in (-1,0,1) for dj in (-1,0,1)])
stone = (x - 25)**2 + (y - 25)**2 < 5**2

# Initial conditions
F = np.ones(shape + (9,)) + 0.01 * np.random.randn(*shape, 9)
F[:,:,7] += 2 * (1 + 0.2 * np.cos(2 * np.pi * x / shape[0] * 4))
F *= rho0 / np.sum(F, 2)[..., None]

print('Evolucionant el fluid...')
for it in tqdm(range(iters)):
    
    # Advection
    for k, (di, dj, w) in enumerate(D2Q9):
        F[:,:,k] = np.roll(F[:,:,k], int(di), axis = 0)
        F[:,:,k] = np.roll(F[:,:,k], int(dj), axis = 1)
    
    # Save stone
    stoneF = F[stone,:]
    
    # Hydro variables
    rho = np.sum(F, 2)
    ux  = np.sum(F * D2Q9[:, 0], 2) / rho
    uy  = np.sum(F * D2Q9[:, 1], 2) / rho
    
    # Collisions
    Feq = rho[..., None] * D2Q9[:, 2] * (1 + 3 * (D2Q9[:, 0] * ux[..., None] + D2Q9[:, 1] * uy[..., None])    +  \
                                         (9/2) * (D2Q9[:, 0] * ux[..., None] + D2Q9[:, 1] * uy[..., None])**2 -  \
                                         (3/2) * (ux[..., None]**2 + uy[..., None]**2))
    F += - (1 / tau) * (F - Feq)
    
    # Apply boundaries 
    F[stone,:] = stoneF[:, ::-1]

    #Compute vorticity
    if it % frame_every == 0:
        dvx_dy = np.roll(ux, -1, axis = 1) - np.roll(ux, 1, axis = 1)
        dvy_dx = np.roll(uy, -1, axis = 0) - np.roll(uy, 1, axis = 0)
        curl[..., it//frame_every] =  dvx_dy - dvy_dx

##### Do the plotting #####
fig2, ax = plt.subplots(figsize = (20,5))
plt.rc('font', size = 12)
plt.close()

stoneshape = 25 + 25j + 6 * np.exp(1j * np.linspace(0, 2*np.pi, 100))
ax.fill(stoneshape.real, stoneshape.imag, color = 'wheat', zorder = 2)
ish = ax.imshow(curl[..., 0].T, cmap='ocean', interpolation = 'bicubic', origin = 'lower', vmin = -.08, vmax = .08)

def plot_flow(frame = 0):
    fig2.set_size_inches(20, 5)
    ish.set_data(curl[..., int(frame)].T)
    return fig2

interact(
    plot_flow,
    frame = FloatSlider(value = 70, min = 0, max = iters//frame_every - 1, step=1,
        description="Temps (unitats arbitràries)",
        style={'description_width': '150px', },
        layout={'width': '800px'},
        readout_format='.2f'),
);


Evolucionant el fluid...


  0%|          | 0/3000 [00:00<?, ?it/s]

interactive(children=(FloatSlider(value=70.0, description='Temps (unitats arbitràries)', layout=Layout(width='…

In [3]:
#@title Espectre de Planck

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

lam = np.linspace(10, 2000, 100)
cmap = mpl.colormaps['jet']
wien = 2.897771955e6

def Planck(lam, T):
    h, c, k = 6.62607015e-34, 2.99792458e8, 1.380649e-23 
    lam_m   = lam * 1e-9
    result  = (2 * h * c**2 / lam_m**5) / (np.exp(h * c / (lam_m * k * T)) - 1)
    return result / 1e13

##### Do the plotting #####
fig3, ax = plt.subplots(figsize = (8, 6))
plt.rc('font', size = 15)
plt.close()

for lam in np.linspace(400, 750, 70):
    ax.plot(2 * [lam], [0.0, 3.5], color = cmap(int( 255 * (lam - 400) / 350 )), alpha = 0.7, zorder = 1)

lam = np.linspace(10, 2000, 100)
ln1, = ax.plot(lam, Planck(lam, 5000), 'k')
ln2, = ax.plot([wien/5000, wien/5000], [0.0, 3.5], 'k--')

ax.set_xlim(0, 2000)
ax.set_ylim(0, 3.5)
ax.set_yticks([])
ax.set_xlabel("Longitud d'ona (nm)")
ax.set_ylabel("Intensitat")

def plot_planck(T):
    fig3.set_size_inches(10, 5)
    ln1.set_data(lam, Planck(lam, T))
    ln2.set_data([wien/T, wien/T], [0.0, 3.5])
    return fig3
    
interact(
    plot_planck,
    T   = FloatSlider(value = 5000, min = 3000, max = 7000, step=100,
        description="Temperatura (Kelvin)",
        style={'description_width': '150px', },
        layout={'width': '800px'},
        readout_format='.2f'),
);


interactive(children=(FloatSlider(value=5000.0, description='Temperatura (Kelvin)', layout=Layout(width='800px…

In [4]:
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

shape        = (500, 500)
xrange       = (-10, 10)
yrange       = (-10, 10)

CFL          = 0.5
tmax         = 20
save_every   = 2

###### Prepare the grids
x = np.linspace(*xrange, shape[0] + 1)[:-1, None]
y = np.linspace(*yrange, shape[1] + 1)[None, :-1]

dx, dy = x[1, 0] - x[0, 0], y[0, 1] - y[0, 0]
dt     = CFL * np.minimum(dx, dy)
iters  = np.int64(tmax // dt) + 1
frames = (iters - 1) // save_every + 1

F         = np.zeros(shape + (2,), dtype=np.float32)
stored    = np.zeros(shape + (frames,))

###### Prepare the lens
speed   = 1
ref_ind = 1.5
R       = 11
lens    = ((x - 5)**2 + y**2 < R**2) * ((x + 15)**2 + y**2 < R**2)
f       = R / (2 * (ref_ind - 1))

###### Wave Equations
def rhs(t, F):
    u, v    = F.transpose(2,0,1)
    d2u_dx2 = (np.roll(u, 1, axis=0) - 2*u + np.roll(u, -1, axis=0)) / dx**2
    d2u_dy2 = (np.roll(u, 1, axis=1) - 2*u + np.roll(u, -1, axis=1)) / dy**2
    lap_u   = d2u_dx2 + d2u_dy2

    dX_dt         = np.zeros_like(F)
    dX_dt[..., 0] = v
    dX_dt[..., 1] = speed**2 / (1 + (ref_ind - 1) * lens)**2 * lap_u

    return dX_dt

###### Runge-Kutta 4th order step
def RK4(t, F):
    k1 = rhs(t, F)
    k2 = rhs(t + 0.5 * dt, F + 0.5 * k1 * dt)
    k3 = rhs(t + 0.5 * dt, F + 0.5 * k2 * dt)
    k4 = rhs(t + dt, F + k3 * dt)
    return t + dt, F + dt * (k1 + 2*k2 + 2*k3 + k4) / 6

###### Boundary Conditions
def apply_boundary(t, F):
    k = 2 * np.pi
    w = k * speed  
    F[:2, :, 0]  =     + np.sin( k * x[:2] - w * t)
    F[:2, :, 1]  = - w * np.cos( k * xrange[0] - w * t)
    F[-2:, :, 0] = 0
    F[-2:, :, 1] = 0

    return t, F

###### Time evolution
print('Evolucionant la llum...')
t = 0.0
for i in tqdm(range(iters)):
    t, F = RK4(t, F)
    t, F = apply_boundary(t, F)
    if i % save_every == 0:
        stored[..., i//save_every] = F[..., 0]

##### Do the plotting #####

fig4, ax = plt.subplots(1, 1, figsize= (6,6))
plt.rc('font', size = 15)
plt.close()

ish = ax.imshow(stored[..., 0].T, cmap = 'RdGy', interpolation = 'bilinear' , origin = 'lower', extent = xrange + yrange, vmin = -1, vmax = 1)
ax.imshow(0 * (x + y).T, cmap = 'cool', alpha = 0.5 * np.float32(lens.T), interpolation = 'bilinear', origin = 'lower', extent = xrange + yrange)
ax.plot([-10, -5, f - 5, -5, -10], [2.7, 2.7, 0, -2.7, -2.7], color = 'yellow')

def plot_lens(frame):
    fig4.set_size_inches(6, 6)
    ish.set_data(stored[..., int(frame)].T)
    return fig4
    
interact(
    plot_lens,
    frame = FloatSlider(value = frames//2, min = 0, max = frames - 1 , step=1,
        description="Temps (u. a.)",
        style={'description_width': '150px', },
        layout={'width': '800px'},
        readout_format='.2f'),
);

Evolucionant la llum...


  0%|          | 0/1001 [00:00<?, ?it/s]

interactive(children=(FloatSlider(value=250.0, description='Temps (u. a.)', layout=Layout(width='800px'), max=…